# Season 20 Election Notes

>**FORBIDDEN KNOWLEDGE WARNING**
>
>This document contains some information that is not currently present on the main site and may be considered spoilers.
>Continue at your own discretion. Some information in this document cannot be shared on the Discord without spoiler tags.

In [ ]:
import pandas
%matplotlib inline
from blaseball_mike.models import *
from blaseball_mike.tables import StatType
import plotly.express as plot
import plotly.io as _pio
import plotly.subplots as subplot
from IPython.display import display, Markdown
from copy import deepcopy

import os
import sys
pdir = os.path.abspath(os.path.join(os.path.dirname(''), os.path.pardir))
sys.path.append(pdir)
from display import *
from blessings import *
sys.path.remove(pdir)

_pio.renderers.default = "notebook_connected"

pies = Team.load_by_name("Philly Pies")

# Fix Attractors
real_pies = deepcopy(pies)
for p in real_pies.lineup + real_pies.rotation:
    p.batting_rating = None
    p.pitching_rating = None
    p.baserunning_rating = None
    p.defense_rating = None

In [ ]:
sim = SimulationData.load()
display(Markdown(f"**Last Updated Season {sim.season}, Day {sim.day}**"))

---
## Decrees

Decrees are chosen by majority votes across all teams. `1` decrees will pass this season.

### Sun 3
**Set Sun. If your Team sweeps all 3 games of a Regular Season series, the final non-loss is worth 2 Wins.**

See below for the number of additional Wins this would have added. It is unknown how this interacts with the current
Turntables/SunSun mess.

In [ ]:
extra_wins = 0
s19_games = Game.load_by_season(season=19, team_id=pies.id)
for i in range(0, len(s19_games), 3):
    series = list(s19_games.values())[i:i+3]
    if series[0].winning_team.id == pies.id and series[1].winning_team.id == pies.id and series[1].winning_team.id == pies.id:
        extra_wins += 1
print(f"Pies would have an extra {extra_wins} Wins")

### Sun 9

**Set Sun. Non-losses on the Week End (every 9th Day) are worth 2 Wins.**

See below for the number of additional Wins this would have added. It is unknown how this interacts with the current
Turntables/SunSun mess.

In [ ]:
extra_wins = 0
for g in s19_games.values():
    if (g.day - 1) % 9 == 8 and g.day <= 99:
        if g.winning_team.id == pies.id:
            extra_wins += 1
print(f"Pies would have an extra {extra_wins} Wins")

### Sun 30

**Set Sun. When a game enters Extra Innings, both Teams will immediately earn 1 Win.**

See below for the number of additional Wins this would have added. It is unknown how this interacts with the current
Turntables/SunSun mess.

In [ ]:
extra_wins = 0
for g in s19_games.values():
    if g.inning > 9:
        extra_wins += 1
print(f"Pies would have an extra {extra_wins} Wins")

___
## Wills

Will are selected for each team by raffle. Each team will draw `2` random wills. The bottom 4 teams in the league by
standings will draw `3`. The chosen wills are random, but wills with more votes are more likely to be selected.

### Equivalent Exchange
**Swap a Player from a Team in the League with an Equivalent (within 2 Stars) Player on your Team.**

See below for the 20 best swap candidates for a particular player, sorted by highest stats in the applicable position.

In [ ]:
league = League.load()
all_teams = list(league.teams.values())
all_active = []
for team in all_teams:
    if team.id != pies.id:
        all_active.extend(team.lineup + team.rotation)
all_total_stars = get_total_stars(all_active)

In [ ]:
for player in pies.lineup:
    total_stars = get_total_stars(player)[player.id]
    valid_total_stars = {k: v for k, v in all_total_stars.items() if total_stars-2 < v < total_stars+2}
    player_matches = [x for x in all_active if x.id in valid_total_stars.keys()]
    player_matches.sort(key=lambda x: x.batting_rating, reverse=True)
    display(Markdown(f"####{player.name}"))
    display(get_stars([player] + player_matches[0:20], include_team=True))
    display(set_heatmap(get_batting_stlats([player] + player_matches[0:20]), maxVal=1.5))

In [ ]:
for player in pies.rotation:
    total_stars = get_total_stars(player)[player.id]
    valid_total_stars = {k: v for k, v in all_total_stars.items() if total_stars-2 < v < total_stars+2}
    player_matches = [x for x in all_active if x.id in valid_total_stars.keys()]
    player_matches.sort(key=lambda x: x.pitching_rating, reverse=True)
    display(Markdown(f"####{player.name}"))
    display(get_stars([player] + player_matches[0:20], include_team=True))
    display(set_heatmap(get_pitching_stlats([player] + player_matches[0:20]), maxVal=1.5))

### Swap
**Swap two Players on your Team's Roster.**

This allows switching the position of any two players, including from the shadows. Because the most complex iteration of
this is shadows swaps, see below for that analysis.

**Batting:**

In [ ]:
shadows = real_pies.shadows
shadows.sort(key=lambda x: x.batting_rating, reverse=True)
best_batting_shadows = shadows[0:4]
worst_batter = sort_lineup(pies, 1)[0]

display(get_stars([worst_batter] + best_batting_shadows))
display(set_heatmap(get_batting_stlats([worst_batter] + best_batting_shadows), maxVal=1))

**Pitching:**

In [ ]:
shadows.sort(key=lambda x: x.pitching_rating, reverse=True)
best_pitching_shadows = shadows[0:4]
worst_pitcher = sort_rotation(pies, 1)[0]

display(get_stars([worst_pitcher] + best_pitching_shadows))
display(set_heatmap(get_pitching_stlats([worst_pitcher] + best_pitching_shadows), maxVal=1))

### Move
**Move a Player on your Team to the desired location on your Roster.**

Move would move a player from any location on the team (including Shadows) to another position, increasing the number
of players in that position. See below for moving various players into different positions.

**Batting**

In [ ]:
bhp = best_hitting_pitcher(real_pies)
bph = best_pitching_hitter(real_pies)
worst_batter = sort_lineup(real_pies, num=1)[0]
worst_pitcher = sort_rotation(real_pies, num=1)[0]

display(Markdown(f"Moving {bhp.name} to the Lineup"))
display(pandas.DataFrame(move_player(pies, bhp, "lineup")))

display(Markdown(f"Moving {worst_batter.name} to the Shadows"))
display(pandas.DataFrame(move_player(pies, worst_batter, "shadows")))

display(Markdown(f"Moving {best_batting_shadows[0].name} to the Lineup"))
display(pandas.DataFrame(move_player(pies, best_batting_shadows[0], "lineup")))

**Pitching**

In [ ]:
display(Markdown(f"Moving {bph.name} to the Rotation"))
display(pandas.DataFrame(move_player(pies, bph, "rotation")))

display(Markdown(f"Moving {worst_pitcher.name} to the Shadows"))
display(pandas.DataFrame(move_player(pies, worst_pitcher, "shadows")))

display(Markdown(f"Moving {best_pitching_shadows[0].name} to the Rotation"))
display(pandas.DataFrame(move_player(pies, best_pitching_shadows[0], "rotation")))

### Item Steal
**Take an Item, repair it, and move it to a Player on your Team.**

This would steal an item from another team and give it to a chosen player on our team. The player receiving the item
will drop a random item if there are no open item slots. See below for a list of all non-base items held by players in
the league.

In [ ]:
players = Player.load_all()
carried_items = []
for p in players.values():
    if getattr(p,"_league_team_id", None) == pies.id:
        continue
    for item in p.items:
        if len(item.adjustments) > 1:
            carried_items.append((p, item))

table = []
for p, i in carried_items:
    mods = [Modification.load_one(a["mod"]).title for a in i.adjustments if a["type"] == 0]
    table.append(pandas.Series({"Wielder": p.name, "Team": p.league_team.nickname, "Max Durability": i.durability, "Modifications": mods}, name=i.name))
pandas.DataFrame(table)

### Shadow Infuse
**Boost a random Stat Category of a Player in your Shadows by 20% to 40%.**

See below for how this would affect various team members at the 2 extreme percentages. This will only affect one player.

#### Batting

In [ ]:
display(Markdown("**20% Change**"))
infuse_batting_table, _, _ = improve_team_batting_table(real_pies, 0.20, shadows=True)
display(infuse_batting_table)
display(set_heatmap(get_batting_stlats(improve_team_batting(real_pies, 0.20, shadows=True)), maxVal=1))

In [ ]:
display(Markdown("**40% Change**"))
infuse_batting_table, _, _ = improve_team_batting_table(real_pies, 0.40, shadows=True)
display(infuse_batting_table)
display(set_heatmap(get_batting_stlats(improve_team_batting(real_pies, 0.40, shadows=True)), maxVal=1))

#### Pitching

In [ ]:
display(Markdown("**20% Change**"))
infuse_pitching_table, _, _ = improve_team_pitching_table(real_pies, 0.20, shadows=True)
display(infuse_pitching_table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(real_pies, 0.20, shadows=True)), maxVal=1))

In [ ]:
display(Markdown("**40% Change**"))
infuse_pitching_table, _, _ = improve_team_pitching_table(real_pies, 0.40, shadows=True)
display(infuse_pitching_table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(real_pies, 0.40, shadows=True)), maxVal=1))

#### Baserunning

In [ ]:
display(Markdown("**20% Change**"))
infuse_baserunning_table, _, _ = improve_team_baserunning_table(real_pies, 0.20, shadows=True)
display(infuse_baserunning_table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(real_pies, 0.20, shadows=True)), maxVal=1))

In [ ]:
display(Markdown("**40% Change**"))
infuse_baserunning_table, _, _ = improve_team_baserunning_table(real_pies, 0.40, shadows=True)
display(infuse_baserunning_table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(real_pies, 0.40, shadows=True)), maxVal=1))

#### Defense

In [ ]:
display(Markdown("**20% Change**"))
infuse_defense_table, _, _ = improve_team_defense_table(real_pies, 0.20, shadows=True)
display(infuse_defense_table)
display(set_heatmap(get_defense_stlats(improve_team_defense(real_pies, 0.20, shadows=True)), maxVal=1))

In [ ]:
display(Markdown("**40% Change**"))
infuse_defense_table, _, _ = improve_team_defense_table(real_pies, 0.40, shadows=True)
display(infuse_defense_table)
display(set_heatmap(get_defense_stlats(improve_team_defense(real_pies, 0.40, shadows=True)), maxVal=1))

### Shadow Revoke
**Revoke a Player from your Team's Shadows.**

This would remove a player from our shadows by moving them to another team, without providing a replacement.

### Reform
**Re-roll an eligible Permanent Modification for a Player on your Team.**

Below is a list of all players with modifications that could be rerolled. It would be better to reroll modifications
with negative effects.

In [ ]:
mod_players = [x for x in pies.lineup + pies.rotation if len(x.perm_attr) > 0]
mod_list = pandas.DataFrame()
for player in mod_players:
    data = pandas.Series({"mods": [x.title for x in player.perm_attr if x.id not in UNREFORMED_MODS]}, name=player.name)
    if len(data["mods"]) == 0:
        continue
    mod_list = mod_list.append(data)
mod_list

### Magnify
**Magnify a Player on your Team. Runs batted in or allowed by this Player are multiplied.**

See below for amount of runs batted in/allowed for each player in the previous season. It is unknown what the mutliplier amount is.

In [ ]:
table = get_batting_stats(pies.lineup, season=19)
table.filter(items=["runs_batted_in"])

In [ ]:
table = get_pitching_stats(pies.rotation, season=19)
table.filter(items=["runs_allowed"])

___
## Blessings

Blessings are selected by raffle across all teams. Each blessing will be awarded to one random team, where teams with
more votes are more likely to be selected.

### TWO FREAKIN BATS
**Two random Players in your Team's Lineup with an available Item slot (empty or broken) will open Handcrafted Aluminum Bat Crates.**

[Two of them](https://www.youtube.com/watch?v=btHpHjabRcc). Aluminum bats increase Ground Friction while reducing
Musclitude and Buoyancy. See below for list of batters with an empty item slot.

In [ ]:
free_items = [x for x in pies.lineup if (x.evolution + 1) - len([x for x in x.items if not x.is_broken]) > 0]
get_stars(free_items)

### Shadow Capes Supply Run
**All Players in your Team's Shadows with an available Item slot will open a Generic Cape Crate.**

Capes add Watchfulness, which is a defense stat. See below for list of shadow players with open item slots

In [ ]:
free_items = [x for x in pies.shadows if (x.evolution + 1) - len([x for x in x.items if not x.is_broken]) > 0]
get_stars(free_items)

### Shots in the Dark
**Call the Negative Alternates (maintaining Star count) for 5 random Players in your Team's Shadows.**

A Negative Alternate is a player reroll (maintaining Combined Star value), while adding the Negative mod which makes
their eDensity Negative. See below for our current shadows, the target is random.

In [ ]:
get_stars(pies.shadows)

### Couldn't Hurt
**Call the Negative Alternates (maintaining Star count) for the 2 worst Players on your Team's Active Roster.**

A Negative Alternate is a player reroll (maintaining Combined Star value), while adding the Negative mod which makes
their eDensity Negative. See below for the 2 worst players on the team.

In [ ]:
worst_pies = sort_overall(pies, num=2)
get_stars(worst_pies)

### Green Light
**One Player on all Teams in your Division will play 50% better in Polarity Plus Weather and 50% worse in Polarity Minus Weather.**

~Random~. Playing worse in Polarity Minus is good if it hits a batter, because you dont want to score when it would result in unruns.

### Sponge Blob
**Give all Players with Ego on your Team's Roster the Sponge modification.**

See below for list of players with any level of Ego. The effect of the Sponge modification is unknown.

In [ ]:
egod = [x for x in pies.lineup + pies.rotation + pies.shadows if any(mod in ("EGO1", "EGO2", "EGO3", "EGO4") for mod in x._perm_attr_ids)]
get_stars(egod)

### Heavy-Handed & Light-Handed
**Your Team becomes Heavy-Handed, making all Items held by Players on the Team positively eDense.**

**Your Team becomes Light-Handed, making all Items held by Players on the Team negatively eDense.**

Items are currently not contributing to eDensity for either players or teams, so the exact effect of these is unknown.

### Strange Attractor
**Recruit a Random Player in the League with the Attractor Modification to your Team.**

See below for list of all active players with Attractor. This will not remove an existing player from our team.

Notes:
* This lists the False Stars, they will play using their player attributes instead.

In [ ]:
attractors = []
for team in all_teams:
    attractors.extend([x for x in team.lineup + team.rotation if "ATTRACTOR" in x._perm_attr_ids])
display(get_stars(attractors, include_team=True))

### Fringe Benefit
**Recruit a Random Player with the Perk Modification to your Team.**

See below for list of all active players with Perk. This will not remove an existing player from our team. Note that
Perk adds to eDensity.

In [ ]:
attractors = []
for team in all_teams:
    attractors.extend([x for x in team.lineup + team.rotation  if "PERK" in x._perm_attr_ids])
display(get_stars(attractors, include_team=True))

### Pitch Runner
**Move the Best Baserunner in your Team's Rotation to a random spot in your Team's Lineup.**

This would move the following player to our lineup.

In [ ]:
temp_pies = pies.rotation.copy()
temp_pies.sort(key=lambda x: x.get_baserunning_rating(), reverse=True)
display(get_stars(temp_pies[0]))
set_heatmap(get_batting_stlats([temp_pies[0]] + pies.lineup), maxVal=1.0)

### Slowpoke
**Move the Worst Baserunner in your Team's Lineup to a random spot in your Team's Rotation.**

This would move the following player to our rotation.

In [ ]:
temp_pies = pies.lineup.copy()
temp_pies.sort(key=lambda x: x.get_baserunning_rating())
display(get_stars(temp_pies[0]))
set_heatmap(get_pitching_stlats([temp_pies[0]] + pies.rotation), maxVal=1.0)

### Stuck in the Freezer
**A Random Player in your Team's Rotation will have their Coldness Boosted by 40%.**

Coldness is currently unknown, though potentially prevents stolen bases. See below for change in player stars and stlats.

In [ ]:
new_team = []
for player in real_pies.rotation:
    new_team.append(player.simulated_copy(buffs={"coldness": 0.4}))
display(get_stars(new_team))
display(set_heatmap(get_pitching_stlats(new_team), maxVal=1))

### Shot Caller
**A Random Player in your Team's Lineup will be 40% better at landing Home Runs in a Big Bucket.**

See below for list of teams that currently have Big Buckets in their Stadium.

In [ ]:
stadiums = Stadium.load_all()
table = pandas.DataFrame()
for s in stadiums.values():
    if "BIG_BUCKET" in s._mods_ids:
        table = table.append(pandas.Series({"Stadium Name": s.name}, name=s.team_id.nickname))
table

### Cool Off
**All Players in your Team's Rotation will gain 20% Coldness but lose 5% Ruthlessness.**

Coldness is unknown, and Ruthlessness is more correlated to increased strikes and fewer balls thrown. See below for
change in team stats.

In [ ]:
new_team = []
for player in real_pies.rotation:
    new_team.append(player.simulated_copy(buffs={"coldness": 0.2, "ruthlessness": -0.05}))
display(set_heatmap(get_pitching_stlats(new_team), maxVal=1))

### Global Guts
**All Players in your Team's Lineup will gain 20% Moxie but lose 5% Thwackability.**

Moxie correlates to better plate discipline (more walks), and thwackability correlates to better contact. See below for
change in team stats.

In [ ]:
new_team = []
for player in real_pies.lineup:
    new_team.append(player.simulated_copy(buffs={"moxie": 0.2, "thwackability": -0.05}))
display(set_heatmap(get_batting_stlats(new_team), maxVal=1))

### Backup or Down
**Flip the eDensity of the most eDense Player in your Team's Shadows.**

This would make the most dense shadows player have the Negative modification, making their eDensity negative.

In [ ]:
temp_pies = pies.shadows.copy()
temp_pies.sort(key=lambda x: x.e_density, reverse=True)
display(f"{temp_pies[0].name} eDensity change: {temp_pies[0].e_density} to {-temp_pies[0].e_density}")
display(f"Total Team eDensity change: {pies.e_density} to {pies.e_density - 2*temp_pies[0].e_density}")

### Complete 180

**Flip the eDensity of the most eDense Player in your Team's Rotation.**

This would make the most dense pitcher have the Negative modification, making their eDensity negative.

In [ ]:
temp_pies = pies.rotation.copy()
temp_pies.sort(key=lambda x: x.e_density, reverse=True)
display(f"{temp_pies[0].name} eDensity change: {temp_pies[0].e_density} to {-temp_pies[0].e_density}")
display(f"Total Team eDensity change: {pies.e_density} to {pies.e_density - 2*temp_pies[0].e_density}")

### Head over Tails

**Flip the eDensity of the most eDense Player in your Team's Lineup.**

This would make the most dense batter have the Negative modification, making their eDensity negative.

In [ ]:
temp_pies = pies.lineup.copy()
temp_pies.sort(key=lambda x: x.e_density, reverse=True)
display(f"{temp_pies[0].name} eDensity change: {temp_pies[0].e_density} to {-temp_pies[0].e_density}")
display(f"Total Team eDensity change: {pies.e_density} to {pies.e_density - 2*temp_pies[0].e_density}")

### Low Five

**Flip the eDensity of the five least eDense Players on your Team's Roster.**

See below for list of players and their eDensity impact.

In [ ]:
temp_pies = pies.lineup.copy() + pies.rotation.copy() + pies.shadows.copy()
temp_pies.sort(key=lambda x: x.e_density)
table = pandas.DataFrame()
for p in temp_pies[0:5]:
    table = table.append(pandas.Series({"eDensity": p.e_density}, name=p.name))
display(table)
density_sum = sum([x.e_density for x in temp_pies[0:5]])
display(f"Total Team eDensity change: {pies.e_density} to {pies.e_density - 2*density_sum}")

### Slow Clap

**The first Player in your Team's Lineup will gain the Slow Build mod.**

Slow Build makes players bat 1% better for each At Bat in a game, resetting at the beginning of each game.

### States of Play

**Give all Elsewhere or Scattered Players on your Team's Roster the Undefined mod.**

Undefined makes a player play 50-100% better while Scattered. See below for list of Elsewhere of Scattered players.

In [ ]:
scattered = [x for x in pies.lineup + pies.rotation + pies.shadows if any(mod in ("SCATTERED", "ELSEWHERE") for mod in x._perm_attr_ids)]
get_stars(scattered)

### Solo Stroll

**The worst Batter in your Team's Lineup will draw a walk on 3 balls instead of 4.**

This would add permanent Walk in the Park to our worst batter, listed below.

In [ ]:
get_stars(worst_batter)

### Targeted Evolution

**A random pitcher and a random hitter on your Team will Advance.**

An Advanced player will gain one evolution level, increasing their minimum star level by one for all stats and raising
their stars to the minimum if they are lower than that. It also allows players to hold an additional item.

## Gifts

Gifts are awarded to teams based on contributions from other teams. Each team creates a Wishlist of items that they
would like, and the top items on the wishlist are awarded during the Latesiesta.

### eDense Infusion

**Ratings Boost! Add the eDense Element to 20% of the non-eDense Items held by Players on your Team.**

It is unknown what item eDensity is or how it effects the sim.

### Collector's Editions

Collectors Editions move a Legendary player to the team. Replicas act as copies of that player and will turn to Dust
after elections.

Special Notes:
* Nagomi has fake Attractor stars, this analysis uses the player attributes.
* Chorby is still unstable, and is likely to get incinerated if possible.
* Chorby replicas have 0 soul, so are safe from consumers

In [ ]:
york = Player.find_by_name("York Silk")
nagomi = Player.find_by_name("Nagomi Mcdaniel")
nagomi.batting_rating = None
nagomi.baserunning_rating = None
nagomi.defense_rating = None
aldon = Player.find_by_name("Aldon Cashmoney")
goodwin = Player.find_by_name("Goodwin Morin")
val = Player.find_by_name("Valentine Games")
pm = Player.find_by_name("Pitching Machine")
chorby = Player.find_by_name("Chorby Soul")

display(Markdown("#### Batting"))
display(set_heatmap(get_batting_stlats([chorby, val, aldon, goodwin, pm] + pies.lineup), maxVal=1.5))

display(Markdown("Adding Chorby Soul"))
display(pandas.DataFrame(move_player(pies, chorby, position='lineup')))

display(Markdown("Adding Valentine Games"))
display(pandas.DataFrame(move_player(pies, val, position='lineup')))

display(Markdown("Adding Aldon Cashmoney"))
display(pandas.DataFrame(move_player(pies, aldon, position='lineup')))

display(Markdown("Adding Goodwin Morin"))
display(pandas.DataFrame(move_player(pies, goodwin, position='lineup')))

display(Markdown("Adding Pitching Machine"))
display(pandas.DataFrame(move_player(pies, pm, position='lineup')))

display(Markdown("#### Pitching"))
display(set_heatmap(get_pitching_stlats([nagomi, chorby] + pies.rotation), maxVal=1.5))

display(Markdown("Adding Nagomi McDaniel"))
display(pandas.DataFrame(move_player(pies, nagomi, position='rotation')))

display(Markdown("Adding Chorby Soul"))
display(pandas.DataFrame(move_player(pies, chorby, position='rotation')))

### Solo Editions

These gifts provide a random player with a benefit for the lateseason and postseason.

* Ambitious: Player will overperform in the postseason.
* Unambitious: Unknown, presumably the opposite of Ambitious.

### Team Editions

These gifts provide a team-wide benefit for the lateseason and postseason.

* Late to the Party: Team will overperform in the Lateseason.
* Early to the Party: Unknown, presumably the opposite of Late to the Party.
* Fireproof: Team will be safe from incinerations. Pies only have 1 Solar Eclipse game in the lateseason.
* Soundproof: Team will be safe from Feedback. Pies have 2 Feedback weather games in the lateseason.
* Gravity: Team will be safe from Reverb. Pies only have 1 Reverb weather game in the lateseason.

### Handcrafted Drops

These will provide Items which have additional stat increases and random special effects.

|Type | Attribute | Stat |
| --- | --- | --- |
| Bat | +Thwackability | Batting |
| Cap | +Unthwackability | Pitching |
| Phone | ??? | ??? |
| Quill | ??? | ??? |

See below for a list of players that can receive them.

The Bats will have Subtractor, converting Runs to UnRuns and vice-versa. The Caps will have Underhanded which will
convert earned Home Runs to UnRuns.

**Bat**

In [ ]:
free_items_lineup = [x for x in pies.lineup if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items_lineup)

**Cap, Quill**

In [ ]:
free_items_rotation= [x for x in pies.rotation if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items_rotation)

**Phone**

In [ ]:
free_items = [x for x in pies.lineup + pies.rotation if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items)

### Soul Patches

**Back by Popular Demand! Give 1 Soul to your least Soul Full player. Repeat 10 times.**

See below for final Soul totals. Low soul is dangerous, as Players will become Redacted if their soul hits 0 and
Consumers will drain 1 soul per chomp.

In [ ]:
players = deepcopy(pies.rotation) + deepcopy(pies.lineup)
for i in range(0, 10):
    players.sort(key=lambda x: x.soul)
    worst = players[0]
    worst.soul += 1
    players = [x for x in players if x.id != worst.id] + [worst]
players.sort(key=lambda x: x.soul)
x = pandas.DataFrame()
for p in players:
    old = [x for x in pies.rotation + pies.lineup if x.id == p.id][0]
    x = x.append(pandas.Series({"Soul Added": p.soul - old.soul,"Resulting Soul": p.soul}, name=p.name))
x

### Bargain Bin

**Give a random Player on your Team the Best Item from the Bargain Bin!**

The Bargain Bin will retrieve one random previously dropped item. Below is a list of all non-base items not currently
held by players.

In [ ]:
all_items = Item.load_all()
rare_items = [x for x in all_items.values() if len(x.adjustments) > 1]
players = Player.load_all()
carried_items = []
for p in players.values():
    carried_items.extend(p.items)
dropped_items = [x for x in rare_items if x not in carried_items]
dropped_items.sort(key=lambda x: len(x.adjustments), reverse=True)

table = []
for i in dropped_items:
    mods = [Modification.load_one(a["mod"]).title for a in i.adjustments if a["type"] == 0]
    table.append(pandas.Series({"Durability": i.durability, "Modifications": mods}, name=i.name))
pandas.DataFrame(table)

---
## Appendix
* [Description of Attributes](https://www.blaseball.wiki/w/Player_Attributes)
* [Stlat Viewer](https://slavfox.space/abslve/?foreboding-kaleidoscope#PHIL)
* [Historical Player Graphs](http://yoori.space/hloroscopes/)
